In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_all = pd.read_csv('D:/AMOGH/yelp_resta')

In [3]:
data_all.columns

Index(['Unnamed: 0', 'review_id', 'user_id', 'business_id', 'stars_x', 'date',
       'text', 'useful', 'funny', 'cool',
       ...
       'Corkage', 'DietaryRestrictions_dairy-free',
       'DietaryRestrictions_gluten-free', 'DietaryRestrictions_vegan',
       'DietaryRestrictions_kosher', 'DietaryRestrictions_halal',
       'DietaryRestrictions_soy-free', 'DietaryRestrictions_vegetarian',
       'AgesAllowed', 'RestaurantsCounterService'],
      dtype='object', length=103)

In [4]:
data_ratings = data_all.groupby('name')['stars_x'].mean()

In [5]:
data_ratings.sort_values(ascending=False).head()

name
"é by José Andrés"                  5.0
"Mo's Race Day Cafe"                5.0
"Mizuya"                            5.0
"Mizu Japanese Sushi Restaurant"    5.0
"Mizestro"                          5.0
Name: stars_x, dtype: float64

In [6]:
# Collaborative Filtering

In [7]:

from surprise import Dataset

In [8]:
from surprise import KNNBasic

In [9]:
data = data_all[['user_id','name','stars_x']]

In [10]:
from surprise import Reader

In [11]:
data_load = Dataset.load_from_df(data,reader=Reader(rating_scale = (1,5)))

In [12]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
knn = KNNBasic(sim_options=sim_options)

In [13]:
threshold = int(.9* len(data_load.raw_ratings))

In [14]:
train= data_load.raw_ratings[:threshold]

In [15]:
test = data_load.raw_ratings[threshold:]

In [16]:
test_ = data_load.construct_testset(test)

In [17]:
train_ = data_load.construct_trainset(train)

In [18]:
knn.train(train_)

C:\Users\amogh\Anaconda2\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
predict = knn.test(test_)

In [20]:
predict

[Prediction(uid='B-ym088HX8oddqYIgJBqTA', iid='"Red Rice"', r_ui=5.0, est=3.715611111111111, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='xFQOMduGADTezup4u6ODnw', iid='"Thai Basil"', r_ui=1.0, est=3.715611111111111, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid='isCNqlU0YzNnSQhuHbKJQQ', iid='"Egg N\' Joe"', r_ui=4.0, est=3.715611111111111, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='ibvANuQ5EfR4djKG7oltfg', iid='"Sushi Mon"', r_ui=5.0, est=3.715611111111111, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid='RSLTG7N0wYJhRTW5CGEm5Q', iid='"Raising Cane\'s"', r_ui=4.0, est=3.715611111111111, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid='ZuAPG1e20BxJz-OzK25Glw', iid='"Union House Bar & Restaurant"', r_ui=4.0, est=3.715611111111111, details={'was_impossible': True, 'reason': 'Use

In [21]:
from collections import defaultdict

In [22]:
def recommendations(predict,top=3):
    top_recs = defaultdict(list)
    for uid,iid,r_ui,est, _ in predict:
        top_recs[uid].append((iid,est))
        
        
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:top]
    
    return top_recs

In [24]:
#recommendations(predict)